## Requerimiento 1: Consolidación y Limpieza de Datos BibTeX

Este notebook implementa un sistema robusto de consolidación de archivos BibTeX de múltiples fuentes, con detección de duplicados y manejo de errores.

### Objetivos:
1. Buscar recursivamente archivos `.bib` en directorios
2. Verificar unicidad de archivos por contenido (hash MD5)
3. Detectar duplicados por título normalizado
4. Consolidar entradas únicas en un solo archivo
5. Exportar duplicados para auditoría
6. Manejo robusto de errores en parsing

### Flujo del Proceso:
```
Búsqueda Recursiva → Verificación de Hash → Ordenamiento Natural → 
Parsing con Entorno Limpio → Normalización de Títulos → 
Detección de Duplicados → Consolidación → Exportación
```

### Características Principales:

#### Detección de Duplicados:
- **Por contenido**: Hash MD5 de archivos
- **Por título**: Normalización inteligente (sin puntuación, minúsculas)
- **Registro detallado**: Origen de cada duplicado

#### Manejo de Errores:
- **Entorno limpio**: Directorio temporal por archivo
- **Try-catch**: Captura errores de parsing
- **Continuación**: Procesa archivos restantes si uno falla

#### Estadísticas:
- Archivos procesados
- Entradas únicas consolidadas
- Duplicados detectados
- Errores encontrados

### Tecnologías Utilizadas:
- **pybtex**: Parsing y escritura de BibTeX
- **hashlib**: Cálculo de hash MD5
- **natsort**: Ordenamiento natural de archivos
- **tempfile**: Directorios temporales
- **re**: Normalización con regex

### Estructura de Datos:

#### Entrada:
```
descargas/
├── ieee/
│   ├── ieee_generative_ai_page_1.bib
│   ├── ieee_generative_ai_page_2.bib
│   └── ...
├── sciencedirect/
│   ├── sciencedirect_page_1.bib
│   └── ...
└── springer/
    └── springer_page_1.bib
```

#### Salida:
```
proyecto/salidas/consolidado.bib    # Archivo consolidado
duplicados/duplicados.bib            # Duplicados detectados
```

### Variables de Entorno:
```python
DOWNLOAD_PATH   # Ruta de archivos .bib a procesar
SALIDA_PATH     # Ruta del archivo consolidado
DUPLICATE_PATH  # Ruta del archivo de duplicados
```

### Implementación del Sistema de Consolidación

Este script implementa 4 funciones principales para consolidar archivos BibTeX de forma robusta.

## Función 1: `process_bibtex_file_with_clean_environment(file_path)`

### Propósito:
Procesa un archivo BibTeX en un entorno aislado para evitar problemas de caché del parser.

### Problema que Resuelve:
```python
# Sin entorno limpio:
parser = bibtex_input.Parser()
bib1 = parser.parse_file("file1.bib")  # OK
bib2 = parser.parse_file("file2.bib")  # Puede usar caché de file1
```

### Solución:
```python
# Con entorno limpio:
temp_dir = tempfile.mkdtemp()           # Directorio temporal único
temp_file = os.path.join(temp_dir, "temp_file.bib")
shutil.copy2(file_path, temp_file)      # Copia a temp
parser = bibtex_input.Parser()          # Parser nuevo
bib_data = parser.parse_file(temp_file) # Parsing limpio
shutil.rmtree(temp_dir)                 # Limpieza
```

### Ventajas:
- Evita conflictos de caché
- Cada archivo se procesa independientemente
- Limpieza automática con `finally`

## Función 2: `normalize_title(title)`

### Propósito:
Normaliza títulos para comparación robusta de duplicados.

### Transformaciones:

#### 1. Eliminación de Caracteres Especiales:
```python
re.sub(r'[^\w\s]', '', title.lower())
```
**Ejemplo**:
```
"Machine Learning: A Survey (2024)" 
→ "machine learning a survey 2024"
```

#### 2. Eliminación de Espacios Múltiples:
```python
re.sub(r'\s+', ' ', normalized).strip()
```

#### 3. Conversión a Minúsculas:
```python
.lower()
```

### Casos de Uso:

| Título Original 1 | Título Original 2 | Normalizado | Duplicado |
|-------------------|-------------------|-------------|-----------|
| "AI: A Survey" | "AI - A Survey" | "ai a survey" | Sí |
| "Machine Learning" | "Machine  Learning" | "machine learning" | Sí |
| "Deep Learning" | "Deep Learning." | "deep learning" | Sí |
| "AI Survey" | "ML Survey" | diferentes | No |

### Limitaciones:
- No detecta sinónimos ("car" vs "automobile")
- No detecta variaciones ("Part I" vs "Part II")
- Sensible a palabras adicionales

## Función 3: `merge_bibtex_files(file_paths, output_path, duplicates_path)`

### Propósito:
Función principal que consolida múltiples archivos BibTeX.

### Algoritmo:

#### Paso 1: Inicialización
```python
merged_db = BibliographyData()      # Base de datos consolidada
processed_titles = {}                # Diccionario de títulos vistos
duplicate_entries = set()            # Set de IDs duplicados
```

#### Paso 2: Procesamiento por Archivo
```python
for file_path in file_paths:
    bib_data = process_bibtex_file_with_clean_environment(file_path)
```

#### Paso 3: Procesamiento por Entrada
```python
for entry_id, entry in bib_data.entries.items():
    # 1. Verificar si tiene título
    if 'title' not in entry.fields:
        merged_db.add_entry(entry_id, entry)
        continue
    
    # 2. Normalizar título
    title = entry.fields['title']
    normalized_title = normalize_title(title)
    
    # 3. Verificar duplicados
    if normalized_title in processed_titles:
        # Es duplicado → guardar en lista
        duplicate_entries.add(entry_id)
        duplicates_list.append(entry_text)
    else:
        # Es único → agregar a consolidado
        merged_db.add_entry(entry_id, entry)
        processed_titles[normalized_title] = (entry_id, file_path)
```

#### Paso 4: Exportación
```python
writer = bibtex_output.Writer()
writer.write_file(merged_db, output_path)           # Consolidado
with open(duplicates_path, "w") as f:
    f.writelines(duplicates_list)                    # Duplicados
```

### Manejo de Errores:
```python
try:
    bib_data = process_bibtex_file_with_clean_environment(file_path)
    # ... procesamiento ...
except Exception as e:
    print(f"Error al procesar {file_path}: {e}")
    # Continúa con el siguiente archivo
```

**Errores comunes**:
- `repeated bibliography entry`: Entrada duplicada dentro del mismo archivo
- `syntax error`: Formato BibTeX inválido
- `FileNotFoundError`: Archivo no existe

## Función 4: `main()`

### Propósito:
Función principal que orquesta todo el proceso.

### Flujo Detallado:

#### 1. Búsqueda Recursiva de Archivos:
```python
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.bib'):
            bibtex_files.append(os.path.join(root, file))
```

#### 2. Ordenamiento Natural:
```python
bibtex_files = natsorted(bibtex_files)
```

**Diferencia**:
```
# Ordenamiento estándar:
['page_1.bib', 'page_10.bib', 'page_2.bib']

# Ordenamiento natural (natsorted):
['page_1.bib', 'page_2.bib', 'page_10.bib']
```

#### 3. Verificación de Unicidad por Hash:
```python
file_hash = hashlib.md5()
with open(file_path, 'rb') as f:
    for chunk in iter(lambda: f.read(4096), b''):
        file_hash.update(chunk)
digest = file_hash.hexdigest()
```

**Ventajas del hash MD5**:
- Detecta archivos idénticos con nombres diferentes
- Rápido (lectura por chunks de 4KB)
- Único por contenido

#### 4. Consolidación:
```python
unique_count, duplicate_count = merge_bibtex_files(
    unique_files, 
    output_path, 
    duplicates_path
)
```

In [1]:
import os
import hashlib
import tempfile
import shutil
import re
from pybtex.database.input import bibtex as bibtex_input
from pybtex.database.output import bibtex as bibtex_output
from pybtex.database import BibliographyData
from natsort import natsorted

def process_bibtex_file_with_clean_environment(file_path):
    """Procesa un archivo BibTeX con un entorno limpio para evitar problemas de caché."""
    # Crear directorio temporal
    temp_dir = tempfile.mkdtemp()
    try:
        # Copiar el archivo a un directorio temporal con un nombre único
        temp_file = os.path.join(temp_dir, f"temp_{os.path.basename(file_path)}")
        shutil.copy2(file_path, temp_file)
        
        # Usar un nuevo parser para cada archivo
        parser = bibtex_input.Parser()
        bib_data = parser.parse_file(temp_file)
        
        return bib_data
    finally:
        # Limpiar el directorio temporal
        shutil.rmtree(temp_dir)

def normalize_title(title):
    """Normaliza un título para facilitar la comparación.
    Elimina espacios extra, signos de puntuación y convierte a minúsculas."""
    if not title:
        return ""
    # Eliminar caracteres especiales y convertir a minúsculas
    normalized = re.sub(r'[^\w\s]', '', title.lower())
    # Eliminar espacios múltiples y convertir a minúsculas
    normalized = re.sub(r'\s+', ' ', normalized).strip().lower()
    return normalized

def merge_bibtex_files(file_paths, output_path, duplicates_path):
    merged_db = BibliographyData()
    duplicates_list = []
    processed_titles = {}  # Cambiado de processed_ids a processed_titles
    duplicate_entries = set()  # Para contar entradas duplicadas únicas

    for file_path in file_paths:
        try:
            print(f"\nProcesando: {file_path}")
            bib_data = process_bibtex_file_with_clean_environment(file_path)

            for entry_id, entry in bib_data.entries.items():
                # Verificar si el entry tiene un campo de título
                if 'title' not in entry.fields:
                    print(f"  Advertencia: Entrada {entry_id} sin título en {file_path}, se agregará como única")
                    merged_db.add_entry(entry_id, entry)
                    continue
                
                # Normalizar el título para comparación
                title = entry.fields['title']
                normalized_title = normalize_title(title)
                
                if normalized_title in processed_titles:
                    # Encontramos un título duplicado
                    original_entry_id, original_file = processed_titles[normalized_title]
                    print(f"  Duplicado encontrado por título: {title}")
                    print(f"  Original ID: {original_entry_id} en: {original_file}")
                    print(f"  Duplicado ID: {entry_id} en: {file_path}")
                    
                    # Agregar el ID a la lista de duplicados únicos
                    duplicate_entries.add(entry_id)

                    # Guardar duplicado como texto en la lista
                    duplicates_list.append(f"@{entry.type}{{{entry_id},\n")
                    duplicates_list.append(f"  title = {{{title}}},\n")
                    for field, value in entry.fields.items():
                        if field != 'title':  # Ya agregamos el título
                            duplicates_list.append(f"  {field} = {{{value}}},\n")
                    duplicates_list.append("}\n\n")
                    
                else:
                    # Es un título nuevo, lo agregamos
                    merged_db.add_entry(entry_id, entry)
                    processed_titles[normalized_title] = (entry_id, file_path)

        except Exception as e:
            print(f"Error al procesar {file_path}: {e}")

    # Crear directorios de salida si no existen
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    os.makedirs(os.path.dirname(duplicates_path), exist_ok=True)
    
    # Guardar el archivo consolidado
    writer = bibtex_output.Writer()
    writer.write_file(merged_db, output_path)
    print(f"Archivo consolidado guardado en: {output_path}")

    # Guardar el archivo de duplicados manualmente
    if duplicates_list:
        with open(duplicates_path, "w", encoding="utf-8") as f:
            f.writelines(duplicates_list)
        print(f"Duplicados guardados en: {duplicates_path}")
    else:
        print("No se encontraron duplicados.")
    
    return len(merged_db.entries), len(duplicate_entries)

def main():
    folder_path = os.getenv("DOWNLOAD_PATH") 

    # Buscar archivos .bib recursivamente en todas las subcarpetas
    bibtex_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.bib'):
                full_path = os.path.join(root, file)
                bibtex_files.append(full_path)
    
    # Ordenamos usando natsorted para lograr un "orden natural"
    bibtex_files = natsorted(bibtex_files)

    # Verificar en que orden se procesaran los archivos
    print("Orden de procesamiento de archivos:")
    for i, f in enumerate(bibtex_files):
        print(f"{i+1}. {f}")

    # Verificar que los archivos sean únicos
    print("\nVerificando unicidad de archivos...")
    file_hashes = {}
    unique_files = []
    
    for file_path in bibtex_files:
        # Calcular hash del archivo
        file_hash = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(4096), b''):
                file_hash.update(chunk)
        
        # Verificar si ya hemos visto este hash
        digest = file_hash.hexdigest()
        if digest in file_hashes:
            print(f"¡ADVERTENCIA! Archivo duplicado detectado:")
            print(f"  - {file_path}")
            print(f"  - {file_hashes[digest]}")
            print(f"  Ambos tienen el mismo hash: {digest}")
        else:
            file_hashes[digest] = file_path
            unique_files.append(file_path)
    
    print(f"Total de archivos encontrados: {len(bibtex_files)}")
    print(f"Archivos únicos por contenido: {len(unique_files)}")
    
    # Proceder solo con archivos únicos
    bibtex_files = unique_files

    # Rutas para archivos de salida
    output_path = os.path.join( os.getenv("SALIDA_PATH") , "consolidado.bib")
    duplicates_path = os.path.join( os.getenv("DUPLICATE_PATH"), "duplicados.bib")
    
    unique_count, duplicate_count = merge_bibtex_files(bibtex_files, output_path, duplicates_path)
    
    print(f"\nResumen:")
    print(f"  Archivos procesados: {len(bibtex_files)}")
    print(f"  Entradas únicas: {unique_count}")
    print(f"  Entradas duplicadas: {duplicate_count}")

if __name__ == "__main__":
    main()

/home/nop/Documentos/proyectoAnalisisAlgoritmos/venv/lib/python3.13/site-packages/pybtex/plugin/__init__.py:26: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Orden de procesamiento de archivos:
1. /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee/ieee_generative_ai_page_1.bib
2. /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee/ieee_generative_ai_page_2.bib
3. /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee/ieee_generative_ai_page_3.bib
4. /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee/ieee_generative_ai_page_4.bib
5. /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee/ieee_generative_ai_page_5.bib
6. /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee/ieee_generative_ai_page_6.bib
7. /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee/ieee_generative_ai_page_7.bib
8. /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee/ieee_generative_ai_page_8.bib
9. /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee/ieee_generative_ai_page_9.bib
10. /home/nop/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee/ieee_generative_ai_page_10.bib
11